In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

import matplotlib.pyplot as plt
%matplotlib inline

import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

In [6]:
from nltk import bigrams, word_tokenize
from nltk.util import ngrams

sentence = "I am a boy."
tokens = word_tokenize(sentence)
tokens

['I', 'am', 'a', 'boy', '.']

In [11]:
bigram = bigrams(tokens)
trigram = ngrams(tokens, 3)

print('bigram: ')
for t in bigram:
    print(t)

print('\ntrigram : ')
for t in trigram:
    print(t)

bigram: 
('I', 'am')
('am', 'a')
('a', 'boy')
('boy', '.')

trigram : 
('I', 'am', 'a')
('am', 'a', 'boy')
('a', 'boy', '.')


In [12]:
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="SS", right_pad_symbol="SE")
for t in bigram :
    print(t)

('SS', 'I')
('I', 'am')
('am', 'a')
('a', 'boy')
('boy', '.')
('.', 'SE')


In [20]:
from nltk import ConditionalFreqDist
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="SS", right_pad_symbol="SE")
cfd = ConditionalFreqDist([(t[0], t[1]) for t in bigram])
cfd

<ConditionalFreqDist with 6 conditions>

In [19]:
cfd.conditions()

['SS', 'I', 'am', 'a', 'boy', '.']

In [21]:
cfd["SS"]

FreqDist({'I': 1})